In [2]:
pip install -q kaggle

In [3]:
from google.colab import files

In [4]:
mkdir ~/.kaggle

In [5]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sanjeevkrishnan","key":"495233389277cd938d8878de0bb52899"}'}

In [6]:
cp kaggle.json ~/.kaggle/

In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets list

ref                                                             title                                       size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -----------------------------------------  -----  -------------------  -------------  ---------  ---------------  
akshaydattatraykhare/diabetes-dataset                           Diabetes Dataset                             9KB  2022-10-06 08:55:25          15242        447  1.0              
akshaydattatraykhare/data-for-admission-in-the-university       Data for Admission in the University         4KB  2022-10-27 11:05:45           3711         84  1.0              
aneesayoub/world-universities-ranking-2022                      World Universities Ranking 2022             41KB  2022-11-17 03:47:37           1049         28  0.9117647        
whenamancodes/covid-19-coronavirus-pandemic-dataset             COVID -19 Coronavirus Pandemic Dataset   

In [9]:
!kaggle datasets download -d arbethi/diabetic-retinopathy-level-detection

100% 9.65G/9.66G [00:50<00:00, 117MB/s]
100% 9.66G/9.66G [00:50<00:00, 205MB/s]


In [10]:
!unzip diabetic-retinopathy-level-detection.zip

Archive:  diabetic-retinopathy-level-detection.zip
  inflating: inception-diabetic.h5   
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cfb17a7cc8d4.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cfdbaef73a8b.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cfed7c1172ec.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cff262ed8f4c.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/cffc50047828.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d02b79fc3200.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d0926ed2c8e5.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d160ebef4117.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d16e39b9d6f0.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d16e59a2b33a.png  
  inflating: preprocessed dataset/preprocessed dataset/testing/0/d18f64

In [11]:
imageSize = [299,299]
trainPath = r"/content/preprocessed dataset/preprocessed dataset/training"
testPath = r"/content/preprocessed dataset/preprocessed dataset/testing"

In [12]:
from tensorflow.keras.layers import Dense,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.applications.xception import Xception,preprocess_input
from glob import glob
import numpy as np
import matplotlib.pyplot as plt


In [13]:
train_datagen = ImageDataGenerator(rescale =1./255,
                                   shear_range= 0.2,
                                   zoom_range= 0.2,
                                   horizontal_flip= True,)
test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
training_set = train_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/training',
                                                target_size = (299,299),
                                                batch_size = 32,
                                                class_mode = 'categorical')



Found 3662 images belonging to 5 classes.


In [15]:
test_set = test_datagen.flow_from_directory('/content/preprocessed dataset/preprocessed dataset/testing',
                                            target_size = (299, 299),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 734 images belonging to 5 classes.


In [16]:
xception = Xception(input_shape=imageSize + [3], weights='imagenet',include_top=False)

83683744/83683744 [==============================] - 1s 0us/step


In [17]:
# don't train existing weights
for layer in xception.layers:
  layer.trainable = False

In [18]:
# our layers - you can add more if you want
x = Flatten()(xception.output)

In [19]:
prediction = Dense(5, activation='softmax')(x)

In [20]:
# create a model object
model = Model(inputs=xception.input, outputs=prediction)

In [21]:
# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

In [22]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [23]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data= test_set,
  epochs=30,
  steps_per_epoch=len(training_set)//32,
  validation_steps=len(test_set)//32
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/30
3/3 [==============================] - 62s 19s/step - loss: 9.9884 - accuracy: 0.3854
Epoch 2/30
3/3 [==============================] - 50s 14s/step - loss: 12.2669 - accuracy: 0.4479
Epoch 3/30
3/3 [==============================] - 49s 15s/step - loss: 11.3369 - accuracy: 0.5938
Epoch 4/30
3/3 [==============================] - 51s 15s/step - loss: 5.7442 - accuracy: 0.6042
Epoch 5/30
3/3 [==============================] - 50s 15s/step - loss: 6.3254 - accuracy: 0.5521
Epoch 6/30
3/3 [==============================] - 48s 15s/step - loss: 4.5788 - accuracy: 0.6458
Epoch 7/30
3/3 [==============================] - 52s 16s/step - loss: 5.0286 - accuracy: 0.6667
Epoch 8/30
3/3 [==============================] - 51s 15s/step - loss: 4.2739 - accuracy: 0.5938
Epoch 9/30
3/3 [==============================] - 50s 15s/step - loss: 2.9265 - accuracy: 0.6875
Epoch 10/30
3/3 [==============================] - 51s 15s/step - loss: 3.8594 - accuracy: 0.7708
Epoch 11/30
3/3 [==========

In [24]:
model.save('Updated-Xception-diabetic-retinopathy.h5')

In [25]:
!tar -zcvf xception-diabetic-retinopathy-final.tgz Updated-Xception-diabetic-retinopathy.h5

Updated-Xception-diabetic-retinopathy.h5


In [26]:
!pip install ibm_watson_machine_learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 25.6 MB/s 
     |████████████████████████████████| 51 kB 704 kB/s 
     |████████████████████████████████| 824 kB 41.2 MB/s 
     |████████████████████████████████| 133 kB 49.9 MB/s 
     |████████████████████████████████| 547 kB 53.1 MB/s 
  Created wheel for ibm-cos-sdk: filename=ibm_cos_sdk-2.7.0-py2.py3-none-any.whl size=72563 sha256=08aa62c0a0b15444cf01e12dccb5d697adcfacfc5b3ab9831ab9f28e9bf380d1
  Stored in directory: /root/.cache/pip/wheels/47/22/bf/e1154ff0f5de93cc477acd0ca69abfbb8b799c5b28a66b44c2
  Created wheel for ibm-cos-sdk-core: filename=ibm_cos_sdk_core-2.7.0-py2.py3-none-any.whl size=501013 sha256=1714cc69ed55c57073436004e991ee678c50285bc00cbd8dd7ab613b761750c1
  Stored in directory: /root/.cache/pip/wheels/6c/a2/e4/c16d02f809a3ea998e17cfd02c13369281f3d232aaf5902c19
  Created wheel for ibm-cos-sdk-s3transfer: filename=ibm_cos

In [27]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
    'url':'https://us-south.ml.cloud.ibm.com',
    'apikey':'qQuCS3JSXjY7QOtoBGf9sItfxbYkWZu3pP4s8zIz343j'
}

client = APIClient(wml_credentials)

Python 3.7 and 3.8 frameworks are deprecated and will be removed in a future release. Use Python 3.9 framework instead.


In [28]:
client

In [29]:
client.spaces.get_details()

{'resources': []}

In [31]:
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  --------------------  ------------------------
ID                                    NAME                  CREATED
cc7b2601-24de-44f7-b474-104d0258bde4  Diabetic retinopathy  2022-11-19T13:09:06.697Z
------------------------------------  --------------------  ------------------------


In [32]:
space_uid = "cc7b2601-24de-44f7-b474-104d0258bde4"

In [33]:
client.set.default_space(space_uid)

'SUCCESS'

In [34]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [35]:
software_space_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [36]:
model_details = client.repository.store_model(model="xception-diabetic-retinopathy-final.tgz", meta_props={
    client.repository.ModelMetaNames.NAME:"Diabetic_Retinopathy_Detection",
    client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid
})

In [37]:
model_details

{'entity': {'hybrid_pipeline_software_specs': [],
  'software_spec': {'id': 'acd9c798-6974-5d2f-a657-ce06e986df4d',
   'name': 'tensorflow_rt22.1-py3.9'},
  'type': 'tensorflow_2.7'},
 'metadata': {'created_at': '2022-11-19T13:12:33.159Z',
  'id': '035fd7e7-efcc-4128-a77d-46b8869fcad8',
  'modified_at': '2022-11-19T13:12:40.937Z',
  'name': 'Diabetic_Retinopathy_Detection',
  'owner': 'IBMid-6620043X6T',
  'resource_key': 'e4951931-7d78-4a32-b739-ce6ec89a2003',
  'space_id': 'cc7b2601-24de-44f7-b474-104d0258bde4'},
 'system': {'warnings': []}}

In [38]:
model_id = client.repository.get_model_id(model_details)
model_id

'035fd7e7-efcc-4128-a77d-46b8869fcad8'

In [39]:
client.repository.download(model_id, 'Diabetic_Retinopathy_Detection.tgz')

Successfully saved model content to file: 'Diabetic_Retinopathy_Detection.tgz'


'/content/Diabetic_Retinopathy_Detection.tgz'